In [6]:
# # Exploración del Modelo DenseNet Pre-entrenado

# ## 1. Introducción
#
# Este notebook carga y explora la arquitectura de un modelo DenseNet guardado en un archivo `.hdf5`. El objetivo es comprender la estructura de la red, sus capas y el número de parámetros antes de utilizarla para la clasificación de radiografías de tórax.
#
# **DenseNet:** Es una arquitectura de Red Neuronal Convolucional (CNN) donde cada capa dentro de un "bloque denso" está conectada directamente con todas las capas posteriores en ese bloque. Esto promueve la reutilización de características, mejora el flujo de gradientes y a menudo permite lograr un buen rendimiento con menos parámetros que otras arquitecturas.
#
# **Archivo `.hdf5`:** Es un formato estándar para guardar modelos entrenados en Keras/TensorFlow. Contiene la arquitectura del modelo, los pesos aprendidos durante el entrenamiento y, opcionalmente, la configuración del optimizador.

# ## 2. Importar Librerías

# In[ ]:
# Import Densenet from Keras
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K
import tensorflow as tf
from tensorflow import keras # Usar tf.keras
import os
import h5py # Para inspeccionar el archivo hdf5 si es necesario

In [7]:
# ## 3. Cargar el Modelo

# In[ ]:
# Definir la ruta al archivo del modelo
# Asumiendo que el notebook está en 'notebooks/' y el modelo en 'models/'
MODEL_DIR = "../models/"
MODEL_FILENAME = "densenet.hdf5" # Asegúrate que este sea el nombre exacto
model_path = os.path.join(MODEL_DIR, MODEL_FILENAME)

print(f"Intentando cargar el modelo desde: {model_path}")

# Cargar el modelo
try:
    # Usar tf.keras.models.load_model
    model = DenseNet121(weights=model_path, include_top=False)
    print("\n¡Modelo cargado exitosamente!")
except FileNotFoundError:
    print(f"\nError: No se encontró el archivo del modelo en '{model_path}'.")
    print("Asegúrate de que el archivo exista y la ruta sea correcta.")
    # Detener la ejecución si el modelo no se carga
    raise
except Exception as e:
    print(f"\nOcurrió un error al cargar el modelo: {e}")
    # Podría ser un problema de versión de TensorFlow/Keras o archivo corrupto
    raise

Intentando cargar el modelo desde: ../models/densenet.hdf5

¡Modelo cargado exitosamente!


In [8]:
## 4. Explorar la Arquitectura del Modelo

# Una vez cargado el modelo, podemos usar el método `.summary()` para ver un resumen detallado de sus capas.

# In[ ]:
# Mostrar el resumen del modelo
print("\n--- Resumen del Modelo DenseNet ---")
model.summary()


--- Resumen del Modelo DenseNet ---
Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, None, None,   0          ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1/conv (Conv2D)            (None, None, None,   9408        ['zero_padding2d[0][0]']         
                                64)                

In [4]:
 ### 4.1. Interpretación del Resumen
#
# Al observar el `model.summary()`, presta atención a:
#
# * **Layers (Tipo):** Los diferentes tipos de capas utilizadas (ej. `InputLayer`, `Conv2D`, `BatchNormalization`, `Activation` (ReLU), `MaxPooling2D`, `Concatenate`, `DenseBlock`, `TransitionBlock`, `GlobalAveragePooling2D`, `Dense`).
# * **Output Shape:** Las dimensiones de la salida de cada capa. Verás cómo cambian las dimensiones espaciales (alto, ancho) y el número de canales (profundidad) a medida que la señal pasa por la red.
# * **Param #:** El número de parámetros entrenables en cada capa.
#     * Las capas convolucionales (`Conv2D`) y densas (`Dense`) tienen la mayoría de los parámetros.
#     * Las capas como `BatchNormalization`, `Activation`, `Pooling`, `Concatenate` generalmente tienen pocos o ningún parámetro entrenable.
# * **Total params:** El número total de parámetros en el modelo.
# * **Trainable params:** El número de parámetros que se actualizarán durante el entrenamiento (si reentrenas o haces fine-tuning).
# * **Non-trainable params:** Parámetros que no se entrenan (a menudo asociados con capas de `BatchNormalization` cuando se usan modelos pre-entrenados y se congelan esas capas).
#
# **Componentes Clave de DenseNet (a buscar en el summary):**
# * **Bloques Densos (Dense Blocks):** Secuencias de capas (usualmente BN -> ReLU -> Conv) donde la salida de cada capa se concatena con la entrada de las capas siguientes dentro del bloque.
# * **Capas de Transición (Transition Layers):** Capas ubicadas entre bloques densos, que típicamente realizan convolución (a menudo 1x1 para reducir canales) y pooling (para reducir dimensiones espaciales).
# * **Concatenación:** La operación clave que combina los mapas de características de capas anteriores.
# * **Capa Final:** Usualmente una capa de `GlobalAveragePooling2D` seguida de una capa `Dense` con el número de neuronas de salida adecuado para tu tarea de clasificación (ej. 14 neuronas con activación sigmoide para clasificación multietiqueta binaria).

# ## 5. (Opcional) Inspeccionar Configuración del Modelo

# El archivo HDF5 también puede contener la configuración con la que se compiló el modelo (optimizador, pérdida, métricas).


In [9]:
# Ver la configuración de compilación (si está guardada)
try:
    config = model.get_config() # Obtiene la configuración de la arquitectura
    # print("\n--- Configuración de la Arquitectura (Primer Nivel) ---")
    # print(config.keys()) # Muestra las claves principales como 'name', 'layers'

    # Intentar acceder a la configuración de compilación (puede no estar siempre)
    print("\n--- Información de Compilación (si está disponible) ---")
    print(f"Optimizador: {model.optimizer}")
    # Acceder al nombre si es un objeto, o imprimir directamente si es None/string
    if hasattr(model.optimizer, 'name'):
         print(f"   Nombre del Optimizador: {model.optimizer.name}")
    elif hasattr(model.optimizer, '_name'): # A veces el nombre está en un atributo protegido
         print(f"   Nombre del Optimizador: {model.optimizer._name}")

    print(f"Función de Pérdida: {model.loss}")
    print(f"Métricas: {model.metrics_names}") # Nombres de las métricas

except AttributeError:
    print("\nEl modelo no parece tener información de compilación guardada (optimizer, loss, metrics).")
except Exception as e:
    print(f"\nError al obtener información de compilación: {e}")


--- Información de Compilación (si está disponible) ---
Optimizador: None

El modelo no parece tener información de compilación guardada (optimizer, loss, metrics).


In [23]:
# --- AGREGACIÓN: Explorar primeras y últimas capas ---
layers_l = model.layers

print("\n--- Primeras 5 Capas ---")
# print(layers_l[0:5]) # Imprime la representación del objeto
for layer in layers_l[0:5]:
    print(f"- Nombre: {layer.name}, Tipo: {type(layer).__name__}")

print("\n--- Últimas 5 Capas ---")
# Imprime las últimas 5 capas reales
for layer in layers_l[-5:]:
    print(f"- Nombre: {layer.name}, Tipo: {type(layer).__name__}")
# --- FIN AGREGACIÓN ---


--- Primeras 5 Capas ---
- Nombre: input_1, Tipo: InputLayer
- Nombre: zero_padding2d, Tipo: ZeroPadding2D
- Nombre: conv1/conv, Tipo: Conv2D
- Nombre: conv1/bn, Tipo: BatchNormalization
- Nombre: conv1/relu, Tipo: Activation

--- Últimas 5 Capas ---
- Nombre: conv5_block16_concat, Tipo: Concatenate
- Nombre: bn, Tipo: BatchNormalization
- Nombre: relu, Tipo: Activation
- Nombre: global_average_pooling2d, Tipo: GlobalAveragePooling2D
- Nombre: dense, Tipo: Dense


In [24]:
# --- AGREGACIÓN: Explorar Capas Convolucionales ---
# Obtener las capas convolucionales y mostrar las primeras 5
conv2D_layers = [layer for layer in model.layers
                 if isinstance(layer, keras.layers.Conv2D)] # Chequeo de tipo más robusto

print("\n--- Primeras 5 Capas Conv2D ---")
for layer in conv2D_layers[0:5]:
     print(f"- Nombre: {layer.name}, Filtros: {layer.filters}, Kernel: {layer.kernel_size}, Strides: {layer.strides}, Padding: {layer.padding}")

# Mostrar el número total de capas convolucionales
print(f"\nTotal de capas Conv2D en el modelo base: {len(conv2D_layers)}")
# --- FIN AGREGACIÓN ---



--- Primeras 5 Capas Conv2D ---
- Nombre: conv1/conv, Filtros: 64, Kernel: (7, 7), Strides: (2, 2), Padding: valid
- Nombre: conv2_block1_1_conv, Filtros: 128, Kernel: (1, 1), Strides: (1, 1), Padding: valid
- Nombre: conv2_block1_2_conv, Filtros: 32, Kernel: (3, 3), Strides: (1, 1), Padding: same
- Nombre: conv2_block2_1_conv, Filtros: 128, Kernel: (1, 1), Strides: (1, 1), Padding: valid
- Nombre: conv2_block2_2_conv, Filtros: 32, Kernel: (3, 3), Strides: (1, 1), Padding: same

Total de capas Conv2D en el modelo base: 120


In [25]:
## 5. Explorar Entrada y Salida del Modelo Base

# In[ ]:
# --- AGREGACIÓN: Explorar Entrada y Salida ---
# Mostrar la entrada esperada por el modelo base
# DenseNet121 pre-entrenada usualmente espera 3 canales (RGB)
print("\n--- Entrada del Modelo Base ---")
# print("Se espera una entrada con 3 canales (formato standard Keras)")
print(f"Forma de Entrada: {model.input_shape}") # Muestra (None, H, W, C)
input_channels = model.input_shape[-1]
print(f"Número de Canales de Entrada: {input_channels}")
if input_channels == 3:
    print("Nota: Si tus imágenes son en escala de grises (1 canal), necesitarás adaptar la primera capa del modelo o duplicar el canal de entrada durante el preprocesamiento.")
elif input_channels == 1:
     print("La entrada ya está configurada para 1 canal (escala de grises).")


# Mostrar la salida del modelo base (antes de añadir nuestra cabecera)
print("\n--- Salida del Modelo Base ---")
x = model.output
print(f"Forma de Salida (Feature Map): {x.shape}") # Muestra (None, h, w, c) -> características extraídas
output_channels = x.shape[-1]
print(f"Número de Canales de Salida (Características): {output_channels}")
# --- FIN AGREGACIÓN ---



--- Entrada del Modelo Base ---
Forma de Entrada: (None, None, None, 3)
Número de Canales de Entrada: 3
Nota: Si tus imágenes son en escala de grises (1 canal), necesitarás adaptar la primera capa del modelo o duplicar el canal de entrada durante el preprocesamiento.

--- Salida del Modelo Base ---
Forma de Salida (Feature Map): (None, 5)
Número de Canales de Salida (Características): 5


In [28]:
# Print the number of output channels
print("The output has 1024 channels")
x = model.output
x

The output has 1024 channels


<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'dense')>

In [17]:
# Add a global spatial average pooling layer
x_pool = GlobalAveragePooling2D()(x)
x_pool

<KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'global_average_pooling2d')>

In [18]:
# Define a set of five class labels to use as an example
labels = ['Emphysema', 
          'Hernia', 
          'Mass', 
          'Pneumonia',  
          'Edema']
n_classes = len(labels)
print(f"In this example, you want your model to identify {n_classes} classes")

In this example, you want your model to identify 5 classes


In [19]:
# Add a logistic layer the same size as the number of classes you're trying to predict
predictions = Dense(n_classes, activation="sigmoid")(x_pool)
print("Predictions have {n_classes} units, one for each class")
predictions

Predictions have {n_classes} units, one for each class


<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'dense')>

In [20]:
# Create an updated model
model = Model(inputs=model.input, outputs=predictions)

In [21]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy')
# (You'll customize the loss function in the assignment!)